In [1]:

%pprint

Pretty printing has been turned OFF


In [45]:

# SET other subtypes as 0; assuming no rows affected if primary and secondary columns are the same
subtypes_list = ['is_task_scope', 'is_minimum_qualification', 'is_preferred_qualification',
                 'is_legal_notification', 'is_job_title', 'is_office_location', 'is_job_duration',
                 'is_supplemental_pay', 'is_educational_requirement', 'is_interview_procedure',
                 'is_corporate_scope', 'is_posting_date', 'is_other']
for primary_column in subtypes_list:
    for secondary_column in subtypes_list:
        CURSOR.execute(su.set_secondary_column0_formatted_sql_str.format(secondary_column, primary_column,
                                                                         secondary_column))

In [41]:

import pandas as pd
%run ../py/html_analysis.py

crf = CrfUtilities()
sql_str = f'''
    SELECT
        np.[navigable_parent],
        pos.[pos_symbol]
    FROM
        [Jobhunting].[dbo].[NavigableParents] np INNER JOIN
        [Jobhunting].[dbo].[PartsOfSpeech] pos ON
            pos.[is_header] = np.[is_header] AND
            pos.[is_task_scope] = np.[is_task_scope] AND
            pos.[is_minimum_qualification] = np.[is_minimum_qualification] AND
            pos.[is_preferred_qualification] = np.[is_preferred_qualification] AND
            pos.[is_legal_notification] = np.[is_legal_notification] AND
            pos.[is_job_title] = np.[is_job_title] AND
            pos.[is_office_location] = np.[is_office_location] AND
            pos.[is_job_duration] = np.[is_job_duration] AND
            pos.[is_supplemental_pay] = np.[is_supplemental_pay] AND
            pos.[is_educational_requirement] = np.[is_educational_requirement] AND
            pos.[is_interview_procedure] = np.[is_interview_procedure] AND
            pos.[is_corporate_scope] = np.[is_corporate_scope] AND
            pos.[is_posting_date] = np.[is_posting_date] AND
            pos.[is_other] = np.[is_other];'''
pos_df = pd.DataFrame(crf.su.get_execution_results(crf.cursor, sql_str, verbose=False))
POS_DICT = pos_df.set_index('navigable_parent')['pos_symbol'].to_dict()

In [42]:

HEADER_PATTERN_DICT = s.load_object('HEADER_PATTERN_DICT')
tuples_list = []
for file_name, feature_dict_list in HEADER_PATTERN_DICT.items():
    total = len(feature_dict_list)
    symbol_count = 0
    for feature_dict in feature_dict_list:
        if feature_dict['child_str'] in POS_DICT:
            symbol_count += 1
    #print(f'{file_name}: {symbol_count}/{total}')
    tuples_list.append((file_name, symbol_count/total))
for file_name, completed_percentage in sorted(tuples_list, key=lambda x: x[1], reverse=True):
    if completed_percentage < 1.0:
        print(file_name, completed_percentage)
        break

Data_Analyst_Data_Scientist_-_Portland,_OR_97202_-_Indeed.com_d9a2e6caf751bd78.html 0.627906976744186


In [43]:

FEATURE_DICT_LIST = HEADER_PATTERN_DICT[file_name]
ZMQInteractiveShell_obj = get_ipython()
su = crf.su
CURSOR = crf.cursor

In [44]:

def get_dictionary_code():
    output_str = '\n'
    for feature_dict in FEATURE_DICT_LIST:
        tag_str = feature_dict['child_str']
        if tag_str not in POS_DICT:
            break
        else:
            tag_str = ''
    if "'" in tag_str:
        tag_str = tag_str.replace('"', '\\"')
        output_str += f'tag_str = "{tag_str}"\n'
    else:
        output_str += f"tag_str = '{tag_str}'\n"
    output_str += "POS_DICT[tag_str] = 'O-xx'\n"
    output_str += 'sql_str = r"""UPDATE NavigableParents\n    SET\n        is_header = 0,\n'
    output_str += '        is_task_scope = 0,\n        is_minimum_qualification = 0,\n'
    output_str += '        is_preferred_qualification = 0,\n        is_legal_notification = 0,\n'
    output_str += '        is_job_title = 0,\n        is_office_location = 0,\n        is_job_duration = 0,\n'
    output_str += '        is_supplemental_pay = 0,\n        is_educational_requirement = 0,\n'
    output_str += '        is_interview_procedure = 0,\n        is_corporate_scope = 0,\n'
    output_str += '        is_posting_date = 0,\n        is_other = 0\n'
    output_str += '    WHERE (navigable_parent LIKE ? ESCAPE \'\\\')"""\n'
    output_str += r"cursor_obj = CURSOR.execute(sql_str, (crf.su.wc_rgx.sub(r'\\\g<1>', tag_str),))"
    output_str += '\nCURSOR.commit()'
    
    return output_str

In [ ]:

ZMQInteractiveShell_obj.set_next_input(text=get_dictionary_code(), replace=True)

In [10]:

pos_explanation_dict = s.load_object('pos_explanation_dict')
pos_explanation_dict

{'H-TS': 'Task Scope Header', 'H-RQ': 'Required Qualifications Header', 'H-PQ': 'Preferred Qualifications Header', 'H-LN': 'Legal Notifications Header', 'H-JT': 'Job Title Header', 'H-OL': 'Office Location Header', 'H-JD': 'Job Duration Header', 'H-SP': 'Supplemental Pay Header', 'H-ER': 'Education Requirements Header', 'H-IP': 'Interview Procedures Header', 'H-CS': 'Corporate Scope Header', 'H-PD': 'Post Date Header', 'H-O': 'Other Header', 'O': 'Non-header'}